In [1]:
!nvidia-smi

Mon Jul 24 05:11:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:02:00.0 Off |                  N/A |
| 34%   75C    P2   112W / 250W |    939MiB / 12288MiB |     98%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:04:00.0 Off |                  N/A |
| 22%   

In [3]:
!git clone https://github.com/HusseinMansourMohd/DeformableDETR.git

Cloning into 'DeformableDETR'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 142 (delta 55), reused 96 (delta 43), pack-reused 0
Receiving objects: 100% (142/142), 5.67 MiB | 10.62 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Checking connectivity... done.


In [9]:
%pip install torch==1.8.0
%pip install numpy
%pip install torchvision
%pip install torchsummary
%pip install matplotlib
%pip install timm
%pip install einops==0.6.1

ERROR: Could not find a version that satisfies the requirement torch==1.8.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.8.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
!cd ./DeformableDETR/models/ops && python setup.py install

running install
/local/musaeed/.conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/local/musaeed/.conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        o

In [2]:
#%cd /content/Deformable-DETR/models/ops/MultiScaleDeformableAttention.egg-info

In [3]:
%cd ./DeformableDETR

/local/musaeed/.conda/cuda/Enhancing_Medvit/DeformableDETR


In [4]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

/local/musaeed/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/local/musaeed/.conda/lib/python3.10/site-packages/torch/_masked/__init__.py:223: UserWarning: Failed to initialize NumPy: No module named 'numpy.core' (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:68.)
  example_input = torch.tensor([[-3, -2, -1], [0, 1, 2]])


1.11.0+cu102


AttributeError: module 'torchvision' has no attribute '__version__'

In [5]:
!pip uninstall torch torchvision -y

Found existing installation: torch 1.11.0
Uninstalling torch-1.11.0:
  Successfully uninstalled torch-1.11.0


In [6]:
import torch
import torchvision

print(torch.__version__)

1.11.0+cu102


AttributeError: module 'torchvision' has no attribute '__version__'

In [7]:
!git clone https://github.com/HusseinMansourMohd/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification

Cloning into 'Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification'...
remote: Enumerating objects: 346, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 346 (delta 21), reused 31 (delta 12), pack-reused 306
Receiving objects: 100% (346/346), 119.03 KiB | 0 bytes/s, done.
Resolving deltas: 100% (190/190), done.
Checking connectivity... done.


In [8]:
import sys
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
import torchvision.datasets as dsets
from torchsummary import summary

ImportError: cannot import name 'Image' from 'PIL' (unknown location)

In [ ]:
!pip install timm
!pip install einops

In [ ]:
%cd /content/
%cd /content/Enhancing-MedViT-Incorporating-Adapter-Modules-for-Improved-Medical-Image-Classification/

In [ ]:
import MedVit_Adapter
print(dir(MedVit_Adapter))

In [ ]:
from MedVit_adapter import MedViT_adapter_small as small


In [ ]:
!pip install medmnist

In [ ]:

import medmnist
from medmnist import INFO 
data_flag = 'retinamnist'
# [tissuemnist , pathmnist, chestmnist, dermamnist, octmnisr ,pnemonismnist , retinamnist, bloodmnist, tissuemnist, organcmist, organs ]
download = True

NUM_EPOCHS = 15
BATCH_SIZE = 15
LR = 0.005

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [ ]:

import torchvision.transforms as transforms
#preprocessing
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image:image.convert('RGB')),
    torchvision.transforms.AugMix(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
]
)
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])
     

In [ ]:
import torch.utils.data as data

# load the data
train_dataset = DataClass(split='train', transform=train_transform, download=download)
val_dataset = DataClass(split='val', transform=train_transform,download=download)
test_dataset = DataClass(split='test', transform=test_transform, download=download)


# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=val_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

In [ ]:

print(train_dataset)
print("+++++++++++++++++")
print(test_dataset)

In [ ]:
model = small()

In [ ]:
model.proj_head[0] = nn.Linear(in_features=1024, out_featuers=n_classes, bais=True)

In [ ]:
#define loss function and optimizer
if task == 'multi-task, binaty-class':
    criterion = nn.BCEWihtLogLoss()
else:
    criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

In [ ]:
criterion

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from torch.nn.functional import softmax
from tqdm import tqdm
# training
for epoch in range(NUM_EPOCHS):

  model = model.to(device)
  model.train()

  for inputs, targets in tqdm(train_loader):
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = model(inputs.to(torch.float32))

    if task == 'multi-label, binary-class':
      targets = targets.to(torch.float32).unsqueeze(1)
    else:
      targets = targets.to(torch.long)
      targets = targets.view(-1)


    predicted_classes = torch.argmax(outputs, dim=1)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    # Delete tensors to free up memory
    del inputs, targets, outputs, predicted_classes
    # Empty the cache to clear up some more memory
    torch.cuda.empty_cache()

In [ ]:

from sklearn.metrics import roc_auc_score, accuracy_score

# switch to evaluation mode
def test(split):
  model.eval()
  # Lists to store actual and predicted values
  actuals = []
  probas = []
  predictions = []

  if split == 'val':
        data_loader = train_loader_at_eval
  else:
        data_loader = test_loader
        
  with torch.no_grad():
      for inputs, targets in tqdm(data_loader):

          inputs, targets = inputs.to(device), targets.to(device)

          if task == 'multi-label, binary-class':
              targets = targets.to(torch.float32).unsqueeze(1)
          else:
              targets = targets.to(torch.long)
              targets = targets.view(-1)

          outputs = model(inputs.to(torch.float32))
          softmax_outputs = softmax(outputs, dim=1)

          _, predicted_classes = torch.max(outputs, 1)

          # Store the actual targets and predicted probabilities
          actuals.extend(targets.cpu().numpy())
          probas.extend(softmax_outputs.detach().cpu().numpy())
          # Probability of positive class
          predictions.extend(predicted_classes.cpu().numpy())
  print('\n')
  auc = roc_auc_score(actuals, probas, multi_class='ovr')
  accuracy = accuracy_score(actuals, predictions)

  print('AUC of the model:', auc)
  print('Accuracy of the model:', accuracy)

print('==> Evaluating...')
test('val')
test('test')